In [1]:
import numpy as np

In [2]:

def distPoint2EpipolarLine(F, p1, p2):
    """ Compute the point-to-epipolar-line distance

       Input:
       - F np.ndarray(3,3): Fundamental matrix
       - p1 np.ndarray(3,N): homogeneous coords of the observed points in image 1
       - p2 np.ndarray(3,N): homogeneous coords of the observed points in image 2

       Output:
       - cost: sum of squared distance from points to epipolar lines
               normalized by the number of point coordinates
    """

    N = p1.shape[1]

    homog_points = np.c_[p1, p2]
    epi_lines = np.c_[F.T @ p2, F @ p1]

    denom = epi_lines[0,:]**2 + epi_lines[1,:]**2
    cost = np.sqrt( np.sum( np.sum( epi_lines * homog_points, axis = 0)**2 / denom) / N)

    return cost


## data input and initialize

In [4]:
import os
dirname = os.path.dirname('/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 6/python/code/')

In [11]:
# Number of 3D points to test
N = 40

# Random homogeneous coordinates of 3-D points
X = np.loadtxt(dirname+"/matlab_X.csv", delimiter = ",")

In [8]:
X

array([[-0.53824  , -0.99609  ,  0.1766   ,  1.8169   ,  0.014169 ,
        -0.40906  ,  1.0004   , -2.1335   ,  2.4787   , -0.85772  ,
        -0.021797 ,  0.66457  , -0.32108  ,  2.0975   , -0.95841  ,
        -0.73658  ,  0.097978 ,  1.3709   ,  0.97763  , -1.9895   ,
        -1.1469   ,  0.31566  ,  0.98557  , -0.0068143, -0.78677  ,
         0.03276  ,  0.51277  , -1.6442   ,  1.0856   ,  0.097482 ,
        -0.88921  , -0.91481  , -0.86132  ,  0.061581 ,  1.0138   ,
        -1.6829   , -0.45697  , -1.1292   ,  0.24122  , -0.39351  ],
       [ 0.86723  , -0.52321  ,  0.75518  , -0.12383  , -0.059551 ,
        -1.2819   , -0.80133  ,  0.40398  ,  1.5391   ,  0.39835  ,
        -0.88006  , -0.53014  ,  2.1458   ,  0.38772  , -2.6689   ,
         1.1147   , -0.84962  ,  0.74102  ,  0.93957  ,  1.6782   ,
        -0.70088  ,  1.2744   ,  1.0236   ,  0.088046 , -2.3368   ,
         0.70411  , -1.4975   ,  1.5138   ,  0.62863  ,  0.50698  ,
        -0.98323  ,  0.034324 , -0.35355  ,  0.

In [10]:
X.shape

(4, 40)

In [9]:
# Simulated scene with error-free correspondances
X[2, :] = X[2, :] * 5 + 10
X[3, :] = 1

In [12]:
# points without noise
P1 = np.array([ [500,   0,      320,    0],
                [0,     500,    240,    0],
                [0,     0,      1,      0]])

P2 = np.array([ [500,   0,      320,    -100],
                [0,     500,    240,    0],
                [0,     0,      1,      0]])

# Image (i.e. projected points)
x1 = P1 @ X
x2 = P2 @ X

In [20]:
x1.shape

(3, 40)

In [22]:
# points with noise
sigma = 1e-1
#  noisy_x1 = x1 + sigma * np.random.randn(*x1.shape)
#  noisy_x2 = x2 + sigma * np.random.randn(*x2.shape)

# If you want to get the same results as matlab users, uncomment those two lines
noisy_x1 = np.loadtxt(dirname + "/matlab_noisy_x1.csv", delimiter = ",")
noisy_x2 = np.loadtxt(dirname + "/matlab_noisy_x2.csv", delimiter = ",")

## Test

In [24]:
p1 = x1
p2 = x2

In [28]:
num_P = p1.shape[1]

In [30]:
A = np.zeros((num_P,9))
for i in range(num_P):
    A[i,:] = np.kron( p1[:,i], p2[:,i] ).T

In [32]:
U,sigma, VT = np.linalg.svd(A,full_matrices=True)
F = VT.T[:,-1].reshape(3,3)

In [60]:
F

array([[ 9.93041474e-20, -2.43148804e-18, -1.74072590e-16],
       [ 2.05129684e-18,  1.82925950e-19, -7.07106781e-01],
       [-5.54920434e-17,  7.07106781e-01, -3.15678631e-15]])

In [61]:
U_F,sigma_F, VT_F = np.linalg.svd(F,full_matrices=True)

In [62]:
U_F

array([[ 4.44089210e-16,  2.22044605e-16,  1.00000000e+00],
       [ 7.30079297e-01, -6.83362437e-01, -6.93889390e-17],
       [ 6.83362437e-01,  7.30079297e-01,  0.00000000e+00]])

In [63]:
sigma_F

array([7.07106781e-01, 7.07106781e-01, 9.93041474e-20])

In [64]:
np.diag(sigma_F)

array([[7.07106781e-01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.07106781e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 9.93041474e-20]])

In [65]:
VT_F

array([[-5.15107048e-17,  6.83362437e-01, -7.30079297e-01],
       [-5.92772865e-17,  7.30079297e-01,  6.83362437e-01],
       [ 1.00000000e+00,  7.84776004e-17,  2.90097181e-18]])

In [66]:
U_F @ np.diag(sigma_F) @ VT_F

array([[ 9.93041474e-20,  3.29217642e-16, -1.21964179e-16],
       [ 2.05129684e-18,  1.66533454e-16, -7.07106781e-01],
       [-5.54920434e-17,  7.07106781e-01, -3.21964677e-15]])

In [40]:
sigma_F[2] = 0

In [41]:
sigma_F

array([0.70710678, 0.70710678, 0.        ])

In [ ]:
sigma_M = np.diag(sigma_F)

In [67]:
F_mod = U_F @ np.diag(sigma_F) @ VT_F

In [68]:
F_mod

array([[ 9.93041474e-20,  3.29217642e-16, -1.21964179e-16],
       [ 2.05129684e-18,  1.66533454e-16, -7.07106781e-01],
       [-5.54920434e-17,  7.07106781e-01, -3.21964677e-15]])

## Def fundamentalEightPoint

In [69]:
import numpy as np

def fundamentalEightPoint(p1, p2):
    """ The 8-point algorithm for the estimation of the fundamental matrix F

     The eight-point algorithm for the fundamental matrix with a posteriori
     enforcement of the singularity constraint (det(F)=0).
     Does not include data normalization.

     Reference: "Multiple View Geometry" (Hartley & Zisserman 2000), Sect. 10.1 page 262.

     Input: point correspondences
      - p1 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 1
      - p2 np.ndarray(3,N): homogeneous coordinates of 2-D points in image 2

     Output:
      - F np.ndarray(3,3) : fundamental matrix
    """

    num_P = p1.shape[1]
    A = np.zeros((num_P, 9))
    for i in range(num_P):
        A[i, :] = np.kron(p1[:, i], p2[:, i]).T
    U, sigma, VT = np.linalg.svd(A, full_matrices=True)
    F = VT.T[:, -1].reshape(3, 3)
    U_F, sigma_F, VT_F = np.linalg.svd(F, full_matrices=True)
    sigma_F[2] = 0
    F_mod = U_F @ np.diag(sigma_F) @ VT_F
    
    return F_mod


## Test code

In [70]:
# Estimate Fundamental Matrix via 8-point algorithm
F  = fundamentalEightPoint(x1, x2)
cost_algebraic = np.linalg.norm( np.sum(x2 * (F @ x1)) ) / np.sqrt(N)
cost_dist_epi_line = distPoint2EpipolarLine(F, x1, x2)

print("")
print('Noise-free correspondences');
print('Algebraic error: %f' % cost_algebraic);
print('Geometric error: %f px' % cost_dist_epi_line);


Noise-free correspondences
Algebraic error: 0.000000
Geometric error: 0.000000 px


In [71]:
# Test with noise
F  = fundamentalEightPoint(noisy_x1, noisy_x2) # This gives bad results!

cost_algebraic = np.linalg.norm( np.sum(noisy_x2 * (F @ noisy_x1), axis=0) ) / np.sqrt(N)
cost_dist_epi_line = distPoint2EpipolarLine(F, noisy_x1, noisy_x2)

print("")
print('Noisy correspondences with 8 Point Algorithm')
print('Algebraic error: %f' % cost_algebraic)
print('Geometric error: %f px' % cost_dist_epi_line)



Noisy correspondences with 8 Point Algorithm
Algebraic error: 8.776260
Geometric error: 320.780422 px
